In [1]:
import requests
import bs4
import pandas as pd
import re  # regular expressions - can find patterns in text, which is useful for parsing website responses 
import numpy as np

In [2]:
url = 'https://finance.yahoo.com/quote/ARNC/financials?p=ARNC'
r = requests.get(url)

In [3]:
soup = bs4.BeautifulSoup(r.text)

In [4]:
# seems like most of the data is in 'span' tags
span_list = soup.find_all('span')
# most of the index for the table is found in the span tags where there is class="Va(m)"
vam_span_list = soup.find_all('span', attrs = {'class': 'Va(m)'})

In [5]:
vam_span_list[:5]

[<span class="Va(m)" data-reactid="75">Total Revenue</span>,
 <span class="Va(m)" data-reactid="92">Cost of Revenue</span>,
 <span class="Va(m)" data-reactid="109">Gross Profit</span>,
 <span class="Va(m)" data-reactid="129">Operating Expense</span>,
 <span class="Va(m)" data-reactid="146">Operating Income</span>]

In [6]:
# it seems most of the data (but not all) fall in this range of 17 between one index and the other
# for example, 75 + 17 = 92

In [7]:
total_revenue_id = 75
cost_of_revenue_id = 92
between_total_and_cost = [x for x in span_list if int(x.get('data-reactid')) > total_revenue_id 
        and int(x.get('data-reactid')) < cost_of_revenue_id]
between_total_and_cost

[<span data-reactid="78">7,277,000</span>,
 <span data-reactid="80">7,277,000</span>,
 <span data-reactid="82">7,442,000</span>,
 <span data-reactid="84">6,824,000</span>,
 <span data-reactid="86">6,661,000</span>]

In [8]:
# we'll use this logic to extract the data

In [9]:
# find the dates on the website
dates = [re.findall(r'\d{2}/\d{2}/\d{4}', str(i)) for i in span_list if len(re.findall(r'\d{2}/\d{2}/\d{4}', str(i))) != 0]
dates = [x[0] for x in dates]
# note that the first column is TTM
dates.insert(0, 'TTM')
dates

['TTM', '12/31/2019', '12/31/2018', '12/31/2017', '12/31/2016']

In [10]:
df_list = []
for i in range(len(vam_span_list)):
    r_id1 = int(vam_span_list[i].get('data-reactid'))
    upper_bound_id = r_id1 + 3 + (len(dates) * 2)
    info = [x for x in span_list if int(x.get('data-reactid')) > r_id1 and int(x.get('data-reactid')) < upper_bound_id]
    id_data_dict = {int(x.get('data-reactid')): x.text for x in info}
    data_list = []
    for j in range(r_id1 + 3, r_id1 + 3 + (len(dates) * 2), 2):
        if j in id_data_dict.keys():
            data_list.append(id_data_dict[j])
        else:
            data_list.append(np.nan)
    df_list.append(data_list)
    
index_names = [x.text for x in vam_span_list]
column_names = dates
pd.DataFrame(df_list, index = index_names, columns = column_names)

,TTM,12/31/2019,12/31/2018,12/31/2017,12/31/2016
Total Revenue,"7,277,000","7,277,000","7,442,000","6,824,000","6,661,000"
Cost of Revenue,"6,270,000","6,270,000","6,549,000","5,866,000","5,602,000"
Gross Profit,"1,007,000","1,007,000","893,000","958,000","1,059,000"
Operating Expense,"643,000","643,000","623,000","693,000","736,000"
Operating Income,"364,000","364,000","270,000","265,000","323,000"
Net Non Operating Interest Income Expense,"-102,000","-102,000","-116,000","-158,000","-91,000"
Other Income Expense,"-85,000","-85,000","87,000","144,000","-64,000"
Pretax Income,"177,000","177,000","241,000","251,000","168,000"
Tax Provision,"-48,000","-48,000","71,000","42,000","13,000"
Net Income Common Stockholders,"225,000","225,000","210,000","209,000","155,000"
